In [ ]:
import pandas as pd
import numpy as py
import matplotlib.pyplot as plt


In [ ]:
# Teammate 0 code begins here
state_name_list = [ "Alabama", "Alaska", "Arizona", "Arkansas", "California",
                    "Colorado", "Connecticut", "DC", "Delaware", "Florida",
                    "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana",
                    "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine",
                    "Maryland", "Massachusetts", "Michigan", "Minnesota", 
                    "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada",
                    "New_Hampshire", "New_Jersey", "New_Mexico", "New_York", "North_Carolina",
                    "North_Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania",
                    "Rhode_Island", "South_Carolina", "South_Dakota", "Tennessee", "Texas",
                    "Utah", "Vermont", "Virginia", "West_Virginia", "Wisconsin", 
                    "Wyoming"
                  ]

# An empty collection of dataframes
crime_by_state = {}


debug = False


print("Importing *.csv files...")

# Base file path we will use for reusable code
base_file_path = "Raw_Data/CrimeStateByState_"

# Import the U.S. crime data totals
file_path = base_file_path + "US" + ".csv"
if (debug):
    print(f"Importing file: {file_path}")
us_totals_df = pd.read_csv(file_path, header=5)

# Iterate over and import the state-by-state crime data
for state in state_name_list:
    file_path = base_file_path + state + ".csv"
    if (debug):
        print(f"Importing file: {file_path}")
    crime_by_state[state] = pd.read_csv(file_path, header=5)

# Import the gun ownership data
file_path = "Raw_Data/gun_ownership_by_state.csv"
if (debug):
    print(f"Importing file: {file_path}")
gun_ownership_df = pd.read_csv(file_path)

print("File import complete.")

# Clean up the footer information that was at the bottom of each crime data .csv file
if debug:
    print("Cleaning up the .csv files...")
    
for state in state_name_list:
    if debug:
        print(f"The current state is {state}...")
    crime_by_state[state] = crime_by_state[state][crime_by_state[state]["Year"].astype(str).str.isnumeric()]
    
if debug:
    print(f"The current state is {state}...")
us_totals_df = us_totals_df[us_totals_df["Year"].astype(str).str.isnumeric()]

if debug:
    print("Cleanup of .csv files is complete.")

In [ ]:
# Reindex the gun ownership data
gun_ownership_df.set_index("State", inplace=True)

gun_ownership_df.head()

In [ ]:
# for state in state_name_list:
#     if debug:
#         print(f"The current state is {state}...")
#     this_state = crime_by_state[state]
#     this_state[this_state["Year"] == "2014"]
#     this_state.head()
#     #gun_ownership_df[state].append()

In [ ]:
# Teammate 0 code ends here

In [ ]:
# Teammate 1 code begins here

In [ ]:
# Teammate 1 code ends here

In [ ]:
# Teammate 2 code begins here

In [ ]:
# Teammate 2 code ends here

In [ ]:
# Teammate 3 code begins here

In [ ]:
# Teammate 3 code ends here